In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, balanced_accuracy_score, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import os

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

In [10]:
cols_to_drop = [
    "label:WillettsSpecific2018_enc",
    "label:WillettsMET2018",
    "label:WillettsMET2018_enc",
    "label:WillettsSpecific2018",
    "pid",
    "window_start",
    "window_end",
    "n_samples",
    "duration_seconds",
    "sex","age_group",
    "label:Walmsley2020_enc",
    "label:Walmsley2020",
    "magnitude_mean"

]

In [4]:
path = '../data/data_processed/participants'
file = 'Participants_all.parquet'

In [5]:
df = pd.read_parquet(os.path.join(path, file))

In [11]:
X = df.drop(columns=cols_to_drop)
y = df["label:Walmsley2020_enc"]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [15]:
y_train.head()

844123     3
588313     3
349454     3
1170909    2
619234     0
Name: label:Walmsley2020_enc, dtype: int64

In [16]:
X_train.loc[:, ['corr_xy', 'corr_xz', 'corr_yz']] =  X_train.loc[:, ['corr_xy', 'corr_xz', 'corr_yz']].fillna(0)


In [17]:
X_train.shape

(1474840, 23)

In [18]:
X_train.isna().sum()

x_mean            0
x_std             0
x_min             0
x_max             0
y_mean            0
y_std             0
y_min             0
y_max             0
z_mean            0
z_std             0
z_min             0
z_max             0
energy_x          0
energy_y          0
energy_z          0
energy_total      0
corr_xy           0
corr_xz           0
corr_yz           0
fft_dom_freq      0
fft_peak_power    0
hour_sin          0
hour_cos          0
dtype: int64

In [19]:
y_train.head()

844123     3
588313     3
349454     3
1170909    2
619234     0
Name: label:Walmsley2020_enc, dtype: int64

In [20]:
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)
X_test  = imputer.transform(X_test)


### Comparando antes e depois do imputer

In [24]:
sm = SMOTE(k_neighbors=2, random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

rf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

In [ ]:
%%time
rf.fit(X_train_res, y_train_res)

### Checando o modelo

In [56]:
y_pred_test = rforest.predict(X_test_imputed)
y_pred_test

array([3, 3, 3, ..., 3, 3, 3], shape=(14940,))

In [57]:
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77      4209
           1       0.04      0.71      0.07        28
           2       0.86      0.84      0.85      5219
           3       0.99      0.97      0.98      5484

    accuracy                           0.86     14940
   macro avg       0.67      0.82      0.67     14940
weighted avg       0.89      0.86      0.87     14940

[[3156  462  591    0]
 [   5   20    3    0]
 [ 759   24 4385   51]
 [  56    2  129 5297]]


In [ ]:
prec, rec, f1, sup = precision_recall_fscore_support(y_test, y_pred, average=None)
print("Per-class P/R/F1:", list(zip(prec, rec, f1, sup)))

In [16]:
### APAGAR OOB DEPOIS. fICA AQUI SO DE CURIOSIDADE. ELE NAO É UM BOM MEIO PARA VALIDACAO EXTERNA
print("OOB Accuracy:", rforest.oob_score_)

OOB Accuracy: 0.852690922235511


In [24]:
classes = sorted(list(set(y_test)))  #classes presentes no y_test (no caso, P001)

#  calcula métricas só para essas classes
precision = precision_score(y_test, y_pred, average=None, labels=classes)
recall    = recall_score(y_test, y_pred, average=None, labels=classes)
f1        = f1_score(y_test, y_pred, average=None, labels=classes)

#  traduzindo as classes
class_names = {
    0: "Light",
    1: "Moderate/Vigorous",
    2: "Sedentary",
    3: "Sleep"
}

#  dataframe final de métricas por classe
df_metrics_test = pd.DataFrame({
    "Classe": classes,
    "Categoria": [class_names[c] for c in classes],
    "Precision": precision,
    "Recall": recall,
    "F1": f1
})

df_metrics_test

NameError: name 'y' is not defined